# 사용자 정의 요약 모델 생성 하기

In [1]:
# import os
# import logging

import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Only log error messages
# tf.get_logger().setLevel(logging.ERROR)

# os.environ["TOKENIZERS_PARALLELISM"] = "false"

KeyboardInterrupt: 

In [1]:
# The percentage of the dataset you want to split as train and test
TRAIN_TEST_SPLIT = 0.1

MAX_INPUT_LENGTH = 1024  # Maximum length of the input to the model
MIN_TARGET_LENGTH = 5  # Minimum length of the output by the model
MAX_TARGET_LENGTH = 128  # Maximum length of the output by the model
BATCH_SIZE = 8  # Batch-size for training our model
LEARNING_RATE = 2e-5  # Learning-rate for training our model
MAX_EPOCHS = 1  # Maximum number of epochs we will train the model for

# This notebook is built on the t5-small checkpoint from the Hugging Face Model Hub
MODEL_CHECKPOINT = "psyche/KoT5-summarization"

In [2]:
import json
from datasets import Dataset, load_dataset

file_path = 'sample.jsonl'

_id, document, summary = [], [], []

with open(file_path, 'r', encoding='utf-8') as file:
    while True:
        line = file.readline()
        if not line: break
        json_data = json.loads(line)
        _id.append(json_data['id'])
        document.append(" ".join(json_data['article_original']))
        summary.append(json_data['abstractive'])

raw_datasets = Dataset.from_dict({'id': _id, 'document': document, 'summary': summary})

print(raw_datasets)

Dataset({
    features: ['id', 'document', 'summary'],
    num_rows: 3012
})


In [ ]:
raw_datasets = raw_datasets.train_test_split(
    train_size=TRAIN_TEST_SPLIT, test_size=TRAIN_TEST_SPLIT
)

# Data Pre-processing
모델을 학습하기 전 허깅 페이스의 Transformers 라이브러리로 MODLE_CHECKPOINT에 저장되어 있는 tokenizer 로딩

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

T5 모델에서는 task를 prefix로 명시해주는 경우가 있다.

In [ ]:
prefix = "요약: "